# Log 01-05-2023

## TODOs
- [x] list existing transformer methods
- [ ] list what need to be added
- [ ] develop Model to PyomoModel
- [ ] learn about the original lark tree structure

## Observations

### 11:12 Existing methods
- string
- value
- symbol_name
- definition
- model_definition
- solve_statement
- assignment
- symbol_definition (taken care of by the aliases for each possibility)
    - index_list
    - set_list
    - parameter_list
    - scalar_list
    - equation_list
    - table_list
    - b_variable_list
    - p_variable_list
    - variable_list

### What do I need to build methods to?
terminal/rule???
- seems to be rule

### Needed
- from statement
    - [x] option_setting
    - [x] if_statement
    - [x] loop_statement
    - [x] abort_statement
    - macro?
    - display?
    - macro_conditional?
        - macro_condition
        - negate
- expression??

### Lark tree structure

basically it has two attributes:
- `data`: its type in the grammar
- `children`: its leaves; either Tree or Token (leaves?)

https://lark-parser.readthedocs.io/en/latest/visitors.html#transformer

### 20:37

Hmm, cannot process the if statement because *the transformer method was not implemented*
- I only finished the model classes

### 20:43 workflow

1. design a class for storing the statement
2. add a method in transformer
3. add a method in Model
3. add an if-statement in the transformer`.start()`
4. for children that you don't know if it is taken care of, try to check other statements involving it

### 21:18 conditional
- assignment: dropped?
- equation: dropped?

### 21:19 `?rule`
- so that the rule will not become a tree; the whole tree structure is flattened
- no need to define a method in transformer; also no need to store them; but to store the rules on the RHS

### (01-06) 21:23 New workflow
- you do not even need a Model class for temporarily storing anything
- you can create a single model and then duplicate it for model statement; and then deactivate constraints
    - by thinking about how gams works itself
- and create subclasses for definitions, and add extra code for creating sets, params, etc in Pyomo
- or **DIRECTLY write string**

## Code

### study the lark tree

In [1]:
from gams2pyomo import GamsParser

# f = './test/gams_basic/variable/binary_v.gms'
# f = './test/gams_basic/assignment/basic.gms'
# f = './test/gams_basic/misc/options.gms'
# f = './test/gams_basic/flow_control/if-single.gms'
# f = './test/gams_basic/flow_control/if-elseif-else.gms'
# f = './test/gams_basic/flow_control/loop.gms'
# f = './test/gams_basic/flow_control/abort.gms'
f = './test/gams_real_problem/tanksize2.gms'

with open(f, 'r') as in_file:
    gp = GamsParser(in_file)

# parse_tree = gp.parse()

# print(parse_tree.pretty())
# parse_tree.__dict__
# parse_tree.children[0].data

# model = gp.transform()

In [2]:
from gams2pyomo.transformer import TreeToModel
parse_tree = gp.parse()
transformer = TreeToModel()
model = transformer.transform(parse_tree)

In [3]:
# dir(transformer)
model._tree

[('LIMROW', '0'),
 ('LIMCOL', '0'),
 ('OPTCA', '1E-09'),
 ('OPTCR', '1E-03'),
 ('RESLIM', '1E+04'),
 ('ITERLIM', '1E+09'),
 ('LP', 'CPLEX'),
 ('NLP', 'SNOPT'),
 ('MIP', 'CPLEX'),
 ('MINLP', 'SCIP'),
 [[set] __p__ ""products"" Tree('data', [Tree('symbol_range', [Token('INTEGER', '1'), Token('INTEGER', '3')])]),
  [set] __n__ ""event points"" Tree('data', [Tree('symbol_range', [Token('INTEGER', '1'), Token('INTEGER', '3')])]),
  [set] __h__ ""number of scenarios"" Tree('data', [Tree('symbol_range', [Token('INTEGER', '1'), Token('INTEGER', '2')])]),
  [set] __subh__ ""num. realizations per uncertain param"" Tree('data', [Tree('symbol_range', [Token('INTEGER', '1'), Token('INTEGER', '2')])])],
 [(subh,subh2,subh3)],
 [(p,pp)],
 [[scalar] __VariableInvestmentCostFactor__ ""variable part of the tank investment cost"" Tree('data', [Tree('symbol_id', [Tree('symbol_id', [Token('INTEGER', '0')]), Token('INTEGER', '3271')])]),
  [scalar] __NumDaysInYear__ ""number of days in a year"" Tree('data',

In [3]:
from pyomo.environ import *

# create all declared model
pyomo_models = {}

for _m in model.model_defs:
    # get name from the model definition
    pyomo_models[_m.name] = ConcreteModel()
    m = pyomo_models[_m.name]

    # check if all constraints are included
    if len(_m.equations) == 1 and _m.equations[0].name.lower == 'all':
        all_eq = True
    else:
        all_eq = False

    # define sets
    for s in model.set():
        # print(s)
        isinstance(s.data.children, list) and len(s.data.children) == 1
        s.data.children[0].data == 'symbol_range'
        _idx_1, _idx_2 = str(s.data.children[0].children[0]), str(s.data.children[0].children[1])

        def sequence_set(idx1, idx2):
            if isinstance(idx1, int):
                return list(range(idx1, idx2 + 1))
            prefix = ''
            for i, c in enumerate(idx1):
                try:
                    int(idx1[i:])
                    prefix = idx1[:i]
                    break
                except ValueError:
                    pass

            try:
                n1 = int(idx1.split(prefix)[1])
                n2 = int(idx2.split(prefix)[1])
                _set = range(n1, n2 + 1)
                return [prefix + str(i) for i in _set]
            except ValueError:
                return list(range(int(idx1), int(idx2) + 1))


        setattr(m, s.symbol.name.upper(), Set(initialize=sequence_set(_idx_1, _idx_2), doc=s.description))

In [2]:
model.parameter()

[[parameter] __MinProductionRate[Token('WORD_IDENTIFIER', 'p')]__ ""lower bound on the production rate in m^3/day"" Tree('data', [Tree('data_value', [Tree('symbol_id', [Token('INTEGER', '1')]), 15.0]), Tree('data_value', [Tree('symbol_id', [Token('INTEGER', '2')]), 15.0]), Tree('data_value', [Tree('symbol_id', [Token('INTEGER', '3')]), 7.0])]),
 [parameter] __MaxProductionRate[Token('WORD_IDENTIFIER', 'p')]__ ""upper bound on the production rate in m^3/day"" Tree('data', [Tree('data_value', [Tree('symbol_id', [Token('INTEGER', '1')]), 50.0]), Tree('data_value', [Tree('symbol_id', [Token('INTEGER', '2')]), 50.0]), Tree('data_value', [Tree('symbol_id', [Token('INTEGER', '3')]), 50.0])]),
 [parameter] __InventoryLowerBound[Token('WORD_IDENTIFIER', 'p')]__ ""lower bound on inventory in m^3"" Tree('data', [Tree('data_value', [Tree('symbol_id', [Token('INTEGER', '1')]), 643.0]), Tree('data_value', [Tree('symbol_id', [Token('INTEGER', '2')]), 536.0]), Tree('data_value', [Tree('symbol_id', [To

In [4]:
# sequence_set('1', '3')
m.pprint()

4 Set Declarations
    H : "number of scenarios"
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    N : "event points"
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    P : "products"
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    SUBH : "num. realizations per uncertain param"
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}

4 Declarations: P N H SUBH
